## Document Processing

In [64]:
import sys
import os
from dotenv import load_dotenv

import oracledb

load_dotenv()

# please update with your username, password, hostname and service_name
username = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
dsn = os.getenv("DSN")

try:
    conn = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!")
except Exception as e:
    print("Connection failed!")
    sys.exit(1)

Connection successful!


In [36]:
try:
    cursor = conn.cursor()

    drop_table_sql = """drop table if exists demo_tab"""
    cursor.execute(drop_table_sql)

    create_table_sql = """create table demo_tab (id number, data clob)"""
    cursor.execute(create_table_sql)

    insert_row_sql = """insert into demo_tab values (:1, :2)"""
    rows_to_insert = [
        (
            1,
            "If the answer to any preceding questions is yes, then the database stops the search and allocates space from the specified tablespace; otherwise, space is allocated from the database default shared temporary tablespace.",
        ),
        (
            2,
            "A tablespace can be online (accessible) or offline (not accessible) whenever the database is open.\nA tablespace is usually online so that its data is available to users. The SYSTEM tablespace and temporary tablespaces cannot be taken offline.",
        ),
        (
            3,
            "The database stores LOBs differently from other data types. Creating a LOB column implicitly creates a LOB segment and a LOB index. The tablespace containing the LOB segment and LOB index, which are always stored together, may be different from the tablespace containing the table.\nSometimes the database can store small amounts of LOB data in the table itself rather than in a separate LOB segment.",
        ),
    ]
    cursor.executemany(insert_row_sql, rows_to_insert)

    conn.commit()

    print("Table created and populated.")
    cursor.close()
except Exception as e:
    print("Table creation failed.")
    cursor.close()
    conn.close()
    sys.exit(1)

Table created and populated.


In [37]:
from langchain_community.document_loaders.oracleai import OracleDocLoader
from langchain_core.documents import Document

"""
# loading a local file
loader_params = {}
loader_params["file"] = "propuesta.pdf"
"""

# loading from a local directory
loader_params = {}
loader_params["dir"] = "./files"

"""
# loading from Oracle Database table
loader_params = {
    "owner": "MYUSER",
    "tablename": "demo_tab",
    "colname": "data",
}
"""

""" load the docs """
loader = OracleDocLoader(conn=conn, params=loader_params)
docs = loader.load()

""" verify """
print(f"Number of docs loaded: {len(docs)}")
# print(f"Document-0: {docs[0].page_content}") # content

Number of docs loaded: 1


In [38]:
docs

[Document(metadata={'SOURCE MIME TYPE': 'application/pdf', 'creation date': '7/14/2025 05:01:14 PM', 'author': 'Secretaría', 'revision date': '7/14/2025 05:01:14 PM', 'Creator': '\rMicrosoft® Word para Microsoft 365', 'publisher': 'Microsoft® Word para Microsoft 365', 'title': 'Formato de carta para la solicitud de evaluador y asesor para trabajo de semestre de industria', '_oid': '689c8f242ff846c871241aa4e554f671', '_file': './files\\propuesta.pdf'}, page_content=' \n\nFormato de carta para la solicitud de evaluador y asesor para trabajo de semestre de industria\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFacultad de Ingeniería\n\n\n\n\n\n\n\nCiudad Universitaria: \n\n\n\nCalle 67 N\n\n\n\no. \n\n\n\n53\n\n\n\n-\n\n\n\n108, bloque 21 • \n\n\n\nDirección de correspondencia: \n\n\n\nCalle 70 No 52\n\n\n\n-\n\n\n\n21\n\n\n\n\n\n\n\nTeléfono: \n\n\n\n219 55 02 • \n\n\n\nFax: \n\n\n\n263 82 82 • \n\n\n\nNit: \n\n\n\n890.980.040\n\n\n\n-\n\n\n\n8 • \n\n\n\nApartado: \n\n\n\n1226 \n\n\n\nhttp

In [39]:
from langchain_community.document_loaders.oracleai import OracleTextSplitter
from langchain_core.documents import Document

"""
# Some examples
# split by chars, max 500 chars
splitter_params = {"split": "chars", "max": 500, "normalize": "all"}

# split by words, max 100 words
splitter_params = {"split": "words", "max": 100, "normalize": "all"}

# split by sentence, max 20 sentences
splitter_params = {"split": "sentence", "max": 20, "normalize": "all"}
"""

# split by default parameters
splitter_params = {"normalize": "all"}

# get the splitter instance
splitter = OracleTextSplitter(conn=conn, params=splitter_params)

list_chunks = []
for doc in docs:
    chunks = splitter.split_text(doc.page_content)
    list_chunks.extend(chunks)

""" verify """
print(f"Number of Chunks: {len(list_chunks)}")
# print(f"Chunk-0: {list_chunks[0]}") # content

Number of Chunks: 36


## VectorStore

In [ ]:
import sys
import os
from dotenv import load_dotenv

import oracledb

load_dotenv()

# please update with your username, password, hostname and service_name
username = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
dsn = os.getenv("DSN")

try:
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!")
except Exception as e:
    print("Connection failed!")
    sys.exit(1)

Connection successful!


In [47]:
from langchain_community.vectorstores import oraclevs
from langchain_community.vectorstores.oraclevs import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import Document
from langchain_community.embeddings import OracleEmbeddings

In [41]:
# Define a list of documents (The examples below are 5 random documents from Oracle Concepts Manual )

documents_json_list = [
    {
        "id": "cncpt_15.5.3.2.2_P4",
        "text": "If the answer to any preceding questions is yes, then the database stops the search and allocates space from the specified tablespace; otherwise, space is allocated from the database default shared temporary tablespace.",
        "link": "https://docs.oracle.com/en/database/oracle/oracle-database/23/cncpt/logical-storage-structures.html#GUID-5387D7B2-C0CA-4C1E-811B-C7EB9B636442",
    },
    {
        "id": "cncpt_15.5.5_P1",
        "text": "A tablespace can be online (accessible) or offline (not accessible) whenever the database is open.\nA tablespace is usually online so that its data is available to users. The SYSTEM tablespace and temporary tablespaces cannot be taken offline.",
        "link": "https://docs.oracle.com/en/database/oracle/oracle-database/23/cncpt/logical-storage-structures.html#GUID-D02B2220-E6F5-40D9-AFB5-BC69BCEF6CD4",
    },
    {
        "id": "cncpt_22.3.4.3.1_P2",
        "text": "The database stores LOBs differently from other data types. Creating a LOB column implicitly creates a LOB segment and a LOB index. The tablespace containing the LOB segment and LOB index, which are always stored together, may be different from the tablespace containing the table.\nSometimes the database can store small amounts of LOB data in the table itself rather than in a separate LOB segment.",
        "link": "https://docs.oracle.com/en/database/oracle/oracle-database/23/cncpt/concepts-for-database-developers.html#GUID-3C50EAB8-FC39-4BB3-B680-4EACCE49E866",
    },
    {
        "id": "cncpt_22.3.4.3.1_P3",
        "text": "The LOB segment stores data in pieces called chunks. A chunk is a logically contiguous set of data blocks and is the smallest unit of allocation for a LOB. A row in the table stores a pointer called a LOB locator, which points to the LOB index. When the table is queried, the database uses the LOB index to quickly locate the LOB chunks.",
        "link": "https://docs.oracle.com/en/database/oracle/oracle-database/23/cncpt/concepts-for-database-developers.html#GUID-3C50EAB8-FC39-4BB3-B680-4EACCE49E866",
    },
]

In [43]:
# Create Langchain Documents

documents_langchain = []

for doc in documents_json_list:
    metadata = {"id": doc["id"], "link": doc["link"]}
    doc_langchain = Document(page_content=doc["text"], metadata=metadata)
    documents_langchain.append(doc_langchain)

In [ ]:
# Ingest documents into Oracle Vector Store using different distance strategies

# When using our API calls, start by initializing your vector store with a subset of your documents
# through from_documents(), then incrementally add more documents using add_texts().
# This approach prevents system overload and ensures efficient document processing.

embedder_params = {
    "provider": "database",
    "model": "ALL_MINILM_L12_V2"
}

embeddings = OracleEmbeddings(
    conn=connection,         # tu conexión oracledb
    params=embedder_params,  # parámetros del modelo
    proxy=None               # opcional, si usas proxy HTTP
)
vector_store_dot = OracleVS.from_documents(
    documents_langchain,
    embeddings,
    client=connection,
    table_name="Documents_DOT",
    distance_strategy=DistanceStrategy.DOT_PRODUCT,
)
vector_store_max = OracleVS.from_documents(
    documents_langchain,
    embeddings,
    client=connection,
    table_name="Documents_COSINE",
    distance_strategy=DistanceStrategy.COSINE,
)
vector_store_euclidean = OracleVS.from_documents(
    documents_langchain,
    embeddings,
    client=connection,
    table_name="Documents_EUCLIDEAN",
    distance_strategy=DistanceStrategy.EUCLIDEAN_DISTANCE,
)

# Ingest documents into Oracle Vector Store using different distance strategies
vector_store_dot_ivf = OracleVS.from_documents(
    documents_langchain,
    embeddings,
    client=connection,
    table_name="Documents_DOT_IVF",
    distance_strategy=DistanceStrategy.DOT_PRODUCT,
)
vector_store_max_ivf = OracleVS.from_documents(
    documents_langchain,
    embeddings,
    client=connection,
    table_name="Documents_COSINE_IVF",
    distance_strategy=DistanceStrategy.COSINE,
)
vector_store_euclidean_ivf = OracleVS.from_documents(
    documents_langchain,
    embeddings,
    client=connection,
    table_name="Documents_EUCLIDEAN_IVF",
    distance_strategy=DistanceStrategy.EUCLIDEAN_DISTANCE,
)


In [ ]:
import gc

def conduct_advanced_searches_optimized(vector_stores):
    query = "How are LOBS stored in Oracle Database"
    filter_criteria = {
    "id": ["cncpt_22.3.4.3.1_P2"],
}  # Filtro exacto por metadata

    for i, vs in enumerate(vector_stores, start=1):
        print(f"\n--- Vector Store {i} Advanced Searches ---")

        # Similarity search sin filtro
        print("\nSimilarity search results without filter:")
        results_no_filter = vs.similarity_search(query, 2)
        print([doc.metadata for doc in results_no_filter])

        # Similarity search con filtro
        print("\nSimilarity search results with filter:")
        results_filter = vs.similarity_search(query, 2, filter=filter_criteria)
        print([doc.metadata for doc in results_filter])

        # Similarity search con score (solo metadatos para ahorrar memoria)
        print("\nSimilarity search with relevance score:")
        results_score = vs.similarity_search_with_score(query, 2)
        print([(doc.metadata, score) for doc, score in results_score])

        # Similarity search con score y filtro
        print("\nSimilarity search with relevance score with filter:")
        results_score_filter = vs.similarity_search_with_score(query, 2, filter=filter_criteria)
        print([(doc.metadata, score) for doc, score in results_score_filter])

        # MMR search reducido (menos fetch_k para menos carga)
        print("\nMax marginal relevance search results:")
        results_mmr = vs.max_marginal_relevance_search(query, 2, fetch_k=5, lambda_mult=0.5)
        print([doc.metadata for doc in results_mmr])

        # MMR con filtro
        print("\nMax marginal relevance search results with filter:")
        results_mmr_filter = vs.max_marginal_relevance_search(
            query, 2, fetch_k=5, lambda_mult=0.5, filter=filter_criteria
        )
        print([doc.metadata for doc in results_mmr_filter])

        # Limpieza para liberar memoria antes de pasar al siguiente vector store
        del results_no_filter, results_filter, results_score, results_score_filter
        del results_mmr, results_mmr_filter
        gc.collect()

# Llamada optimizada: procesa uno a uno
vector_store_list = [
    vector_store_dot,
    vector_store_max,
    vector_store_euclidean,
    vector_store_dot_ivf,
    vector_store_max_ivf,
    vector_store_euclidean_ivf
]

conduct_advanced_searches_optimized(vector_store_list)



NameError: name 'vector_store_dot' is not defined